In [67]:
import os
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path
from itertools import combinations

In [65]:
ROOT_DIR = Path().resolve().parent
ROOT_DIR

WindowsPath('C:/Users/90004698/PT. Medco E&P Indonesia/CODE/03_non-business-demand/06_AHP_Tools')

In [66]:
def convert_keys_to_int(obj):
    if isinstance(obj, dict):
        return {int(k) if k.isdigit() else k: v for k, v in obj.items()}
    
    return obj

path_RI_file = f'{ROOT_DIR}/data_static//Ratio_Inconsistency.json'

with open(path_RI_file, 'r') as f:
    RI = json.load(f, object_hook=convert_keys_to_int)

RI

{1: 0,
 2: 0,
 3: 0.58,
 4: 0.9,
 5: 1.12,
 6: 1.24,
 7: 1.32,
 8: 1.41,
 9: 1.45,
 10: 1.49,
 11: 1.51,
 12: 1.48,
 13: 1.56,
 14: 1.57,
 15: 1.59}

In [4]:
variable = [
    'Resource Area',
    'Permeability',
    'Surface and Subsurface Risk'
]

In [6]:
comb = combinations(variable, 2)
init_score = {}

for item in comb:
    init_score[item] = 2

init_score

{('Resource Area', 'Permeability'): 2,
 ('Resource Area', 'Surface and Subsurface Risk'): 2,
 ('Permeability', 'Surface and Subsurface Risk'): 2}

In [13]:
('Permeability', 'Resource Area') in list(init_score.keys())

False

In [22]:
df_score = {}

for val1 in variable:
    for val2 in variable:
        if val1 != val2:
            if (val1, val2) in list(init_score.keys()):
                df_score[(val1, val2)] = init_score[(val1, val2)]
            else:
                df_score[(val1, val2)] = 1 / init_score[(val2, val1)]
        else:
            df_score[(val1, val1)] = 1


# Convert the dictionary into a Series and Convert to a DataFrame matrix form
df_score_final = pd.Series(df_score).unstack()
df_score_final.loc['Total'] = df_score_final.sum(axis=0)
df_score_final

,Permeability,Resource Area,Surface and Subsurface Risk
Permeability,1.0,0.5,2.0
Resource Area,2.0,1.0,2.0
Surface and Subsurface Risk,0.5,0.5,1.0
Total,3.5,2.0,5.0


In [24]:
df_score_norm = df_score_final.div(df_score_final.loc['Total'], axis=1)
df_score_norm['Total'] = df_score_norm.sum(axis=1) / len(variable)
df_score_norm

,Permeability,Resource Area,Surface and Subsurface Risk,Total
Permeability,0.285714,0.25,0.4,0.311905
Resource Area,0.571429,0.50,0.4,0.490476
Surface and Subsurface Risk,0.142857,0.25,0.2,0.197619
Total,1.000000,1.00,1.0,1.000000


In [37]:
df_score_calc = df_score_final.copy().iloc[0:len(variable), :].T
df_score_calc['Total'] = df_score_norm['Total']
df_score_calc

,Permeability,Resource Area,Surface and Subsurface Risk,Total
Permeability,1.0,2.0,0.5,0.311905
Resource Area,0.5,1.0,0.5,0.490476
Surface and Subsurface Risk,2.0,2.0,1.0,0.197619


In [46]:
df_eigen = {}
eigen_value = 0

for val in variable:
    df_eigen[val] = float(np.sum(df_score_calc[val] * df_score_calc['Total']))
    eigen_value += float(np.sum(df_score_calc[val] * df_score_calc['Total']))

df_eigen['Eigenvalue Maximum'] = eigen_value
df_eigen['CI (Consistency Index)'] = (eigen_value - len(variable)) / (len(variable) - 1)
df_eigen['CR (Consistency Ratio)'] = df_eigen['CI (Consistency Index)'] / RI[len(variable)]
pd.Series(df_eigen)

Resource Area                  1.509524
Permeability                   0.952381
Surface and Subsurface Risk    0.598810
Eigenvalue Maximum             3.060714
CI (Consistency Index)         0.030357
CR (Consistency Ratio)         0.052340
dtype: float64